In [23]:
import os

import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model

from attention import Attention
from load import load_data

tra_date = '2007-02-14'
val_date = '2015-01-02'
tes_date = '2016-01-04'
model_selection = 'dtml'

index_data_path = './data/kdd17/index/'
data_path = './data/kdd17/processed/'
num_tickers = 50
params = {
    'input_dim': 11,
    'time_steps': 15,
    'epochs': 5,
}

fnames = [fname for fname in os.listdir(data_path) if
            os.path.isfile(os.path.join(data_path, fname))]
print(len(fnames), ' tickers')

stock_layers = [x[:-4] for x in fnames]


stock_context_vector = np.zeros([50, 1970, 11], dtype=float)
index_context_vector = np.zeros([1, 1970, 11], dtype=float)

def time_axis_attention(data_path, fname):
    print(fname)
    tra_pv, tra_wd, tra_gt, val_pv, val_wd, val_gt, tes_pv, tes_wd, tes_gt = load_data(data_path, fname,
            tra_date, val_date, tes_date, seq=params['time_steps']
        ) 
    model_input = Input(shape=(params['time_steps'], params['input_dim']))
    x = Dense(params['input_dim'])(model_input)
    x = LSTM(64, return_sequences=True)(x)
    x = Attention(params['input_dim'])(x)
    # x = Dense(1, activation = 'sigmoid')(x)
    # x = Dense(1)(x)
    # model = Model(model_input, x)
    # model.compile(loss='mae', optimizer='adam')
    # model.summary()

    # save context vector
    # intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    # intermediate_output = intermediate_layer_model(tra_pv)
    # stock_context_vector[idx] = intermediate_output.numpy()

    # model.fit(tra_pv, tra_gt, epochs=params['epochs'])

49  tickers


In [24]:
index = time_axis_attention(index_data_path, 'SPY_processed.csv')

for idx, fname in enumerate(fnames):
    stock_layers[idx] = time_axis_attention(data_path, fname)

    

SPY_processed.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2517


IndexError: index 2517 is out of bounds for axis 0 with size 2517

In [18]:
fnames = [fname for fname in os.listdir(data_path) if
            os.path.isfile(os.path.join(data_path, fname))]
print(len(fnames), ' tickers')


for idx, fname in enumerate(fnames):
    print(fname)
    tra_pv, tra_wd, tra_gt, val_pv, val_wd, val_gt, tes_pv, tes_wd, tes_gt = load_data(data_path, fname,
            tra_date, val_date, tes_date, seq=params['time_steps']
        ) 
    model_input = Input(shape=(params['time_steps'], params['input_dim']))
    x = Dense(params['input_dim'])(model_input)
    x = LSTM(64, return_sequences=True)(x)
    x = Attention(params['input_dim'])(x)
    # x = Dense(1, activation = 'sigmoid')(x)
    # x = Dense(1)(x)
    model = Model(model_input, x)
    model.compile(loss='mae', optimizer='adam')
    model.summary()

    # save context vector
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    intermediate_output = intermediate_layer_model(tra_pv)
    stock_context_vector[idx] = intermediate_output.numpy()

    model.fit(tra_pv, tra_gt, epochs=params['epochs'])


50  tickers
BA.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 15, 11)]          0         
                                                                 
 dense_10 (Dense)            (None, 15, 11)            132       
                                                                 
 lstm_10 (LSTM)              (None, 15, 64)            19456     
                                                                 
 attention_10 (Attention)    (None, 11)                5504      
                                                                 
Total params: 25,092
Trainable params: 25,092
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5


2022-07-17 22:20:46.576893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-17 22:20:46.686189: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2022-07-17 22:20:46.838494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-17 22:20:46.939639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 3s 18ms/step - loss: 0.5198
Epoch 2/5
62/62 [==============================] - 1s 16ms/step - loss: 0.4966
Epoch 3/5
62/62 [==============================] - 1s 16ms/step - loss: 0.4788
Epoch 4/5
62/62 [==============================] - 1s 16ms/step - loss: 0.4787
Epoch 5/5
62/62 [==============================] - 1s 17ms/step - loss: 0.4787
TM.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 15, 11)]          0         
                                                                 
 dense_11 (Dense)            (None, 15, 11)            132       
                                                                 
 lstm_11 (LSTM)              (None, 15, 64)          

2022-07-17 22:20:53.334049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-17 22:20:53.437392: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2022-07-17 22:20:53.590121: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-17 22:20:53.696597: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 2s 17ms/step - loss: 0.4906
Epoch 2/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4901
Epoch 3/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4900
Epoch 4/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4900
Epoch 5/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4900
MO.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 15, 11)]          0         
                                                                 
 dense_12 (Dense)            (None, 15, 11)            132       
                                                                 
 lstm_12 (LSTM)              (None, 15, 64)          

...

 1/62 [..............................] - ETA: 1:47 - loss: 0.6903

2022-07-17 22:26:27.132396: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 3s 22ms/step - loss: 0.5112
Epoch 2/5
62/62 [==============================] - 1s 16ms/step - loss: 0.5091
Epoch 3/5
62/62 [==============================] - 1s 16ms/step - loss: 0.5096
Epoch 4/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4978
Epoch 5/5
62/62 [==============================] - 1s 16ms/step - loss: 0.4888
JNJ.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
Model: "model_120"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_60 (InputLayer)       [(None, 15, 11)]          0         
                                                                 
 dense_59 (Dense)            (None, 15, 11)            132       
                                                                 
 lstm_59 (LSTM)              (None, 15, 64)        

2022-07-17 22:26:33.718258: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-17 22:26:33.820918: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2022-07-17 22:26:33.986640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-17 22:26:34.208680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


62/62 [==============================] - 3s 23ms/step - loss: 0.5293
Epoch 2/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4795
Epoch 3/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4681
Epoch 4/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4680
Epoch 5/5
62/62 [==============================] - 1s 15ms/step - loss: 0.4680


In [19]:
multi_level_context_vector

array([[[-3.94268474e-03, -7.68327503e-04, -7.77846617e-06, ...,
          2.55864183e-03,  6.68808771e-03, -5.28394943e-03],
        [-3.91988689e-03, -1.85969948e-05,  2.78514519e-04, ...,
          1.62527629e-03,  5.63550927e-03, -4.83782729e-03],
        [-3.79534112e-03,  1.00350915e-03,  9.82391415e-04, ...,
          5.01917093e-04,  4.61288402e-03, -4.38891398e-03],
        ...,
        [-1.74346031e-03,  7.89974071e-03,  4.27981466e-03, ...,
          3.17770580e-04, -2.28511728e-03, -3.51359812e-03],
        [-2.32273655e-04,  7.64224492e-03,  4.28253319e-03, ...,
         -1.69997814e-03, -5.04839048e-03, -1.16537570e-03],
        [ 1.41247991e-03,  6.23681024e-03,  3.40810209e-03, ...,
         -2.51824432e-03, -7.17436662e-03,  1.18894898e-03]],

       [[-3.23717296e-03, -3.93352332e-03, -9.29951435e-04, ...,
          3.58164334e-03,  9.75792296e-03, -3.78302950e-03],
        [-3.66728404e-03, -3.00392136e-03,  1.05192745e-03, ...,
          1.20060355e-03,  7.98861310e

In [126]:
# get context vector for index
tra_pv, tra_wd, tra_gt, val_pv, val_wd, val_gt, tes_pv, tes_wd, tes_gt = load_data(data_path, fname,
        tra_date, val_date, tes_date, seq=params['time_steps']
    ) 
model_input = Input(shape=(params['time_steps'], params['input_dim']))
x = Dense(params['input_dim'])(model_input)
x = LSTM(64, return_sequences=True)(x)
x = Attention(params['input_dim'])(x)
# x = Dense(1, activation = 'sigmoid')(x)
# x = Dense(1)(x)
model = Model(model_input, x)
model.compile(loss='mae', optimizer='adam')
model.summary()

# save context vector
intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
intermediate_output = intermediate_layer_model(tra_pv)
multi_level_context_vector[idx] = intermediate_output.numpy()

model.fit(tra_pv, tra_gt, epochs=params['epochs'])

[]
(1, 3)
(2, 3)
[[1. 2. 3.]
 [4. 5. 0.]]


In [128]:
arr = np.empty((0,3))
print(arr)
arr = np.append(arr, np.array([[1, 2, 3]]), axis=0)
arr = np.append(arr, np.array([[4, 5, 0]]), axis=0)
print(arr)

[]
[[1. 2. 3.]
 [4. 5. 0.]]


In [53]:
np.zeros([2,3])

array([[0., 0., 0.],
       [0., 0., 0.]])